In [8]:
import json
import os
import requests
from datetime import datetime
import fal_client
from anthropic import Anthropic

import streamlit as st
import pandas as pd
import json
import zipfile
import os
import ast 
from io import BytesIO
from datetime import datetime
import os
from dotenv import load_dotenv  
from Utilites import *
from runbook import *
import pandas as pd
load_dotenv()  
import requests
from requests.auth import HTTPBasicAuth

ANTHROPIC_API_KEY=os.getenv('ANTHROPIC_API_KEY')  
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')  
PERPLEXITY_API_KEY = os.getenv('PERPLEXITY_API_KEY')   
FAL_KEY = os.getenv('FAL_KEY')

# Initialize Anthropic client
client = anthropic.Anthropic(
    api_key=os.environ.get("ANTHROPIC_API_KEY")
)

def optimize_prompt(prompt, context):
    input_text = f"""You are tasked with optimizing a prompt for DALLE image generation to create proper images for a blog post. Your goal is to refine the given prompt while considering the context of a previous image.

                    First, you will be provided with two inputs:
                    <prompt>{prompt}</prompt>
                    This is the initial prompt for DALLE image generation.

                    <content>"{context}"</content>
                    This represents the context from the previous image.

                    Your Key Focus should be of generating prompts for realistic images

                    To optimize the prompt for DALLE, follow these guidelines:

                    1. Analyze the given prompt and the previous image context.
                    2. Don't make the image too similar to the previous image.
                    3. Ensure the prompt is clear, specific, and descriptive.
                    4. Include relevant details from the previous image context to maintain consistency.
                    5. Use vivid and precise language to describe the desired image.
                    6. Incorporate artistic elements like style, mood, and composition if appropriate.
                    7. Avoid any potential copyright issues or explicit content.

                    Remember that DALLE has a token limit of 1000. To handle this:
                    - Keep your optimized prompt concise and within the token limit.
                    - Prioritize the most important elements of the image description.
                    - Remove any unnecessary words or repetitive information.

                    Provide your optimized prompt within <optimized_prompt> tags. After the optimized prompt, include a brief explanation of your changes and reasoning within <explanation> tags.

                    Your response should follow this format:

                    <optimized_prompt>
                    [Your optimized prompt here]
                    </optimized_prompt>
                    """

    response = client.messages.create(
        model="claude-3-sonnet-20240229",
        max_tokens=4000,
        temperature=0.7,
        system="You are a prompt engineer who generates ONLY realistic Images. Images as close to real and human as possible",
        messages=[
            {
                "role": "user",
                "content": input_text
            }
        ]
    )
    
    # Extract the optimized prompt from the response
    try:
        # Get the content from the response
        response_content = response.content[0].text
        
        # Look for content between <optimized_prompt> tags
        start_tag = "<optimized_prompt>"
        end_tag = "</optimized_prompt>"
        start_index = response_content.find(start_tag) + len(start_tag)
        end_index = response_content.find(end_tag)
        
        if start_index >= 0 and end_index >= 0:
            optimized_prompt = response_content[start_index:end_index].strip()
            return optimized_prompt
        else:
            # If tags aren't found, return the original prompt
            print("Tags not found in response, returning original prompt")
            return prompt
    except Exception as e:
        print(f"Error extracting optimized prompt: {str(e)}")
        return prompt
def download_image(url, filename):
    """Download image from URL and save it to file"""
    response = requests.get(url)
    if response.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(response.content)
        return True
    return False

def generate_image(prompt, identifier):
    """Generate image using fal_client and save it"""
    def on_queue_update(update):
        if isinstance(update, fal_client.InProgress):
            for log in update.logs:
                print(log["message"])

    print(f"prompt: {prompt}")

    

    # Generate optimized prompt
    optimized_prompt = optimize_prompt(prompt, "Blogs/Organization/images/742f4e9d/content-5720236e.png")
    print(f"Optimized prompt: {optimized_prompt}")

    # Generate image using fal_client
    result = fal_client.subscribe(
        "fal-ai/flux-pro/v1.1",
        arguments={
            "prompt": f"{optimized_prompt}. Ultra Realistic images"
        },
        with_logs=True,
        on_queue_update=on_queue_update,
    )

    # Get image URL
    image_url = result['images'][0]['url']
    
    # Create regenerated folder if it doesn't exist
    if not os.path.exists('regenerated'):
        os.makedirs('regenerated')

    # Generate filename with timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"regenerated/{identifier}_{timestamp}.png"

    # Download and save image
    if download_image(image_url, filename):
        print(f"Successfully saved image: {filename}")
    else:
        print(f"Failed to download image for prompt: {identifier}")

    return filename

def process_prompts_file(file_path):
    """Process prompts from JSON file"""
    try:
        with open(file_path, 'r') as file:
            prompts_dict = json.load(file)

        results = {}
        for identifier, prompt in prompts_dict.items():
            print(f"\nProcessing prompt for {identifier}")
            try:
                filename = generate_image(prompt, identifier)
                results[identifier] = {
                    'prompt': prompt,
                    'generated_file': filename
                }
            except Exception as e:
                print(f"Error processing prompt {identifier}: {str(e)}")
                results[identifier] = {
                    'prompt': prompt,
                    'error': str(e)
                }

        # Save results to log file
        with open('generation_results.json', 'w') as f:
            json.dump(results, f, indent=4)

    except Exception as e:
        print(f"Error processing prompts file: {str(e)}")

# Example usage
if __name__ == "__main__":
    # Example prompts file path
    prompts_file = "prompts.json"
    
    # Process the prompts
    process_prompts_file(prompts_file)


Processing prompt for it8_5
prompt: A therapist performing a gentle massage on a patient's back, with visible relaxation and relief on the patient's face
Optimized prompt: A professional massage therapist gently kneading the back muscles of a client lying face down on a massage table, the client's facial expression conveying deep relaxation and relief from tension, the scene set in a tranquil spa or wellness center with warm lighting and soothing decor.
Successfully saved image: regenerated/it8_5_20250104_200309.png

Processing prompt for nl7_3
prompt: A person sitting at a pottery wheel, intensely focused on shaping a clay vessel. The scene should convey a sense of therapeutic engagement and emotional release through the tactile process of working with clay.
Optimized prompt: A serene middle-aged woman with a focused expression sits at a pottery wheel in a sunlit studio. Her hands gently mold a smooth, earthen clay vessel as she finds solace in the rhythmic motion. The scene exudes a

In [ ]:
{
    "it8_5":"A therapist performing a gentle massage on a patient's back, with visible relaxation and relief on the patient's face",
    "nl7_3":"A person sitting at a pottery wheel, intensely focused on shaping a clay vessel. The scene should convey a sense of therapeutic engagement and emotional release through the tactile process of working with clay.",
    "nl8_4":"A collage of people doing gentle exercises like yoga, tai chi, and stretching, with emphasis on proper posture and alignment.",
    "de8_1":"Close-up of hands performing acupuncture on a patient's back, with needles visible and a calm, clinical setting",
    "es8_1":"Close-up of acupuncture needles being applied to a person's back, with a calm and serene expression on their face, in a clean, professional setting.",
    "es8_5":
    " A professional massage therapist performing a therapeutic massage on a client, focusing on areas commonly affected by chronic pain such as the back, neck, and shoulders, in a calming spa-like setting. "
}

In [6]:
from bs4 import BeautifulSoup

def generate_toc(file_path, output_path):
    # Read the HTML file
    with open(file_path, "r", encoding="utf-8") as file:
        html_content = file.read()

    # Parse the HTML content
    soup = BeautifulSoup(html_content, "html.parser")

    # Find all headings (h1, h2, h3, ...)
    headings = soup.find_all(["h1", "h2", "h3", "h4", "h5", "h6"])
    
    # Generate TOC structure
    toc_html = '<ul>\n'
    for heading in headings:
        # Ensure the heading has an id attribute, or create one
        if not heading.has_attr("id"):
            heading_id = heading.text.strip().lower().replace(" ", "-")
            heading["id"] = heading_id
        else:
            heading_id = heading["id"]
        
        # Add the heading to the TOC
        toc_html += f'    <li><a href="#{heading_id}">{heading.text.strip()}</a></li>\n'
    toc_html += '</ul>\n'

    # Insert the TOC into the HTML (at the start of the body)
    body = soup.find("body")
    if body:
        toc_element = BeautifulSoup(toc_html, "html.parser")
        body.insert(0, toc_element)

    # Save the updated HTML to a new file
    with open(output_path, "w", encoding="utf-8") as file:
        file.write(str(soup))

    print(f"TOC generated and saved to {output_path}")

# Provide the input and output file paths
input_file = "html_details"  # Replace with your file path
output_file = "html_with_toc.html"  # Replace with desired output path

generate_toc(input_file, output_file)


TOC generated and saved to html_with_toc.html


In [5]:
import streamlit as st
import pandas as pd
import json
import zipfile
import os
import ast 
from io import BytesIO
from datetime import datetime
import os
from dotenv import load_dotenv  
from Utilites import *
from runbook import *
import pandas as pd
load_dotenv()  
import requests
from requests.auth import HTTPBasicAuth

ANTHROPIC_API_KEY=os.getenv('ANTHROPIC_API_KEY')  
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')  
PERPLEXITY_API_KEY = os.getenv('PERPLEXITY_API_KEY')   
FAL_KEY = os.getenv('FAL_KEY')


import random
import ast

def read_input_file(file_path: str) -> Dict:
    """Read and parse the input JSON file."""
    with open(file_path, 'r') as file:
        return json.load(file)

def get_random_values_from_string(dictionary_string, num_values):
  try:
    dictionary = ast.literal_eval(dictionary_string)
  except (SyntaxError, ValueError):
    print("Invalid dictionary string format.")
    return {}  # Return an empty dictionary in case of invalid input

  num_values_to_extract = min(num_values, len(dictionary))

  # Create a list of randomly selected keys
  random_keys = random.sample(list(dictionary.keys()), num_values_to_extract)

  # Create a new dictionary with the selected key-value pairs
  random_dict = {key: dictionary[key] for key in random_keys}
  new_keys=map(lambda x: x.replace('-',' ').title(),list(random_dict.keys()))
  random_dict1 = dict(zip(new_keys, random_dict.values()))

  return random_dict1

def GenerateBlogStructure(topic,
                          course_name,
                          course_link,
                          results_blog_dict,
                          results_product_dict,
                          num_of_words_in_blog,
                          num_of_images_in_blog,
                          num_of_headings,
                          blog_percent,
                          promotion_percent,
                          num_of_infographics,
                          number_of_case_studies,
                          inpiration_blog_dict,
                          number_of_tables,
                          num_of_outbound_links,
                          language
                          ):
    blog_content = generate_blog_post(topic,
                                      course_name,
                                      course_link,
                                      results_blog_dict,
                                      results_product_dict,
                                      num_of_words_in_blog,
                                      num_of_images_in_blog,
                                      num_of_headings,
                                      blog_percent,
                                      promotion_percent,
                                      num_of_infographics,
                                      number_of_case_studies,
                                      inpiration_blog_dict.values(),
                                      number_of_tables,
                                      num_of_outbound_links,
                                      language
                                      )


    blog_content=add_content_id(blog_content)

    structure_path='Blogs/Organization/html/blog_structure/'



    blog_identifier=save_html(blog_content,structure_path)


    return blog_identifier

# def SaveImages(blog_identifier):
#     structure_path='Blogs/Organization/html/blog_structure/'
#     with open(f"{structure_path}{blog_identifier}.html") as f:
#           blog_content=f.read()

#     prompts = extract_dalle_prompts(blog_content)
#     images= generate_images(prompts, "Create Ultra Realistic images, No cartoonish images")


#     image_path=f'Blogs/Organization/images/{blog_identifier}/'
#     json_path=f'Blogs/Organization/json/{blog_identifier}.json'

#     image_dict = create_image_dict(blog_content, images)
#     image_path_dict = save_images(image_dict, image_path)
#     save_image_paths(image_path_dict, json_path)

# def ProduceFinalOutput(blog_identifier,website):
#     json_path=f'Blogs/Organization/json/{blog_identifier}.json'
#     structure_path='Blogs/Organization/html/blog_structure/'
#     with open(f"{structure_path}{blog_identifier}.html") as f:
#           blog_content=f.read()
#     updated_html = update_html_with_images(blog_content, json_path,website)
#     print("Add styling")
#     # final_html = add_styling(updated_html)
#     # final_html=updated_html.replace('<img','<figure')
#     updated_html=create_formatted_html(updated_html)
#     output_path='Blogs/Organization/html/final_output/'
#     save_html(updated_html,output_path,blog_identifier)

def SaveImages(blog_identifier):
    structure_path='Blogs/Organization/html/blog_structure/'
    with open(f"{structure_path}{blog_identifier}.html") as f:
          blog_content=f.read()

    prompts = extract_dalle_prompts(blog_content)
    images= generate_images(prompts, "Create Ultra Realistic images, No cartoonish images")

    image_titles=generate_titles_for_prompts(prompts,client)

    image_path=f'Blogs/Organization/images/{blog_identifier}/'
    json_path=f'Blogs/Organization/json/{blog_identifier}.json'

    title_dict,image_dict = create_image_dict(blog_content, images, image_titles)
    image_path_dict = save_images(image_dict, image_path,title_dict)
    save_image_paths(image_path_dict, json_path)

def ProduceFinalOutput(blog_identifier,website):
    json_path=f'Blogs/Organization/json/{blog_identifier}.json'
    structure_path='Blogs/Organization/html/blog_structure/'
    with open(f"{structure_path}{blog_identifier}.html") as f:
          blog_content=f.read()
    updated_html = update_html_with_images(blog_content, json_path,website)
    print("Add styling")
    # final_html = add_styling(updated_html)
    # final_html=updated_html.replace('<img','<figure')
    updated_html=create_formatted_html(updated_html)
    output_path='Blogs/Organization/html/final_output/'
    save_html(updated_html,output_path,blog_identifier)

def DeleteImage(blog_identifier, Content_id):
    image_path = f'Blogs/Organization/images/{blog_identifier}/{Content_id}.png'

    width = height = None  # Initialize width and height

    try:
        if os.path.exists(image_path):
            # Get image size before deleting
            with Image.open(image_path) as img:
                width, height = img.size

            os.remove(image_path)
            print(f"Image deleted successfully: {image_path}")
        else:
            print(f"Image not found: {image_path}")
    except Exception as e:
        print(f"Error deleting image: {e}")

    return width, height


cloudinary.config(
    cloud_name = "ddzaqiihn",
    api_key = "626813739546286",
    api_secret = "MXztcr6KdWsM7XnYGLdBeXegnIw"
)


def AddImage(source_path, blog_identifier, Content_id, width=None, height=None):
    destination_path = f'Blogs/Organization/images/{blog_identifier}/{Content_id}.png'

    try:
        # Ensure the destination directory exists
        os.makedirs(os.path.dirname(destination_path), exist_ok=True)

        # Upload the image to Cloudinary
        upload_result = cloudinary.uploader.upload(source_path)

        # Generate the Cloudinary URL with transformations
        transformation = []
        if width and height:
            transformation = [
                {'width': width, 'height': height, 'crop': 'fill'}
            ]

        resized_url = cloudinary.utils.cloudinary_url(
            upload_result['public_id'],
            transformation=transformation
        )[0]

        # Download the image from Cloudinary
        response = requests.get(resized_url)
        if response.status_code == 200:
            # Save the image content to the destination path
            with open(destination_path, 'wb') as f:
                f.write(response.content)

            print(f"Image added successfully: {destination_path}")
            return resized_url
        else:
            print(f"Failed to download image from Cloudinary. Status code: {response.status_code}")

    except FileNotFoundError:
        print(f"Source file not found: {source_path}")
    except PermissionError:
        print(f"Permission denied. Unable to upload the file.")
    except Exception as e:
        print(f"Error adding image: {e}")

    return None


def UpdateBlogStructure(html_identifier, content_id, updated_text):
    html_file_path=f'Blogs/Organization/html/blog_structure/{html_identifier}.html'
    # Read HTML from the identifier (file path)
    with open(html_file_path, 'r', encoding='utf-8') as file:
        html_content = file.read()

    # Parse the HTML content
    soup = BeautifulSoup(html_content, 'html.parser')

    # Find the element with the specified content_id
    element = soup.find(content_id=content_id)

    if element:
        # Update the text content of the element
        element.string = updated_text

        # Write the updated HTML back to the file
        with open(html_file_path, 'w', encoding='utf-8') as file:
            file.write(str(soup))

        print(f"HTML updated successfully. Content with ID '{content_id}' has been updated.")
    else:
        print(f"Element with ID '{content_id}' not found in the HTML.")

from PIL import Image, ImageDraw, ImageFont
import textwrap
from typing import Tuple
import numpy as np
import PIL

def get_multiline_text_size(text, font, draw):
    """Calculate the width and height of multi-line text."""
    lines = text.split("\n")
    max_width = 0
    total_height = 0

    for line in lines:
        bbox = draw.textbbox((0, 0), line, font=font)
        line_width = bbox[2] - bbox[0]
        line_height = bbox[3] - bbox[1]
        max_width = max(max_width, line_width)
        total_height += line_height

    return max_width, total_height

def create_simple_overlay(image_path, output_path, title, subtitle,
                          margin_ratio=0.1,
                          box_fill_color=(93, 108, 50, 153),
                          border_color="white",
                          border_thickness=2,
                          inner_margin_ratio=0.05):
    # Load the image
    background = Image.open(image_path).convert("RGBA")
    img_width, img_height = background.size

    # Calculate the margins
    margin_x = int(img_width * margin_ratio)
    margin_y = int(img_height * margin_ratio)

    # Calculate square size based on available height
    square_size = img_height - (2 * margin_y)

    # Coordinates for the square box
    left = margin_x
    top = margin_y
    right = left + square_size
    bottom = top + square_size

    # Calculate inner border coordinates (5% inside)
    border_offset = int(square_size * inner_margin_ratio)
    inner_left = left + border_offset
    inner_top = top + border_offset
    inner_right = right - border_offset
    inner_bottom = bottom - border_offset

    # Draw the overlay
    overlay = Image.new("RGBA", background.size, (255, 255, 255, 0))
    draw = ImageDraw.Draw(overlay)

    # Draw main box
    draw.rectangle([left, top, right, bottom], fill=box_fill_color)

    # Draw inner border
    draw.rectangle([inner_left, inner_top, inner_right, inner_bottom],
                  outline=border_color, width=border_thickness)

    # Set initial font size
    font_path = "Merriweather-Bold.ttf"
    title_font_size = 20
    subtitle_font_size = 20
    title_font = ImageFont.truetype(font_path, title_font_size)
    subtitle_font = ImageFont.truetype(font_path, subtitle_font_size)

    # Adjust the text area to fit within the inner border
    text_area_width = inner_right - inner_left
    text_area_height = inner_bottom - inner_top

    # Dynamically adjust the title font size for the new text area
    while True:
        title_width, title_height = get_multiline_text_size(title, title_font, draw)
        if title_width <= text_area_width * 0.9 and title_height <= text_area_height * 0.6:
            title_font_size += 2
            title_font = ImageFont.truetype(font_path, title_font_size)
        else:
            title_font_size -= 2
            title_font = ImageFont.truetype(font_path, title_font_size)
            break

    # Dynamically adjust the subtitle font size
    while True:
        subtitle_width, subtitle_height = get_multiline_text_size(subtitle, subtitle_font, draw)
        if subtitle_width <= square_size * 0.9 and subtitle_height <= square_size * 0.3:
            subtitle_font_size += 2
            subtitle_font = ImageFont.truetype(font_path, subtitle_font_size)
        else:
            subtitle_font_size -= 2
            subtitle_font = ImageFont.truetype(font_path, subtitle_font_size)
            break
    # Center text within the square
    title_width, title_height = get_multiline_text_size(title, title_font, draw)
    subtitle_width, subtitle_height = get_multiline_text_size(subtitle, subtitle_font, draw)

    # Define spacing between title and subtitle (adjust this value as needed)
    title_subtitle_spacing = square_size * 0.1  # 10% of square size for spacing

    # Adjust font sizes
    # Make subtitle smaller relative to title
    while True:
        subtitle_width, subtitle_height = get_multiline_text_size(subtitle, subtitle_font, draw)
        if subtitle_width <= square_size * 0.8:  # Making subtitle width smaller than title
            break
        subtitle_font_size -= 2
        subtitle_font = ImageFont.truetype(font_path, subtitle_font_size)

    # Ensure subtitle is proportionally smaller than title
    if subtitle_font_size > title_font_size * 0.6:  # Subtitle will be 60% of title size
        subtitle_font_size = int(title_font_size * 0.6)
        subtitle_font = ImageFont.truetype(font_path, subtitle_font_size)
        subtitle_width, subtitle_height = get_multiline_text_size(subtitle, subtitle_font, draw)

    # Calculate text positions with new spacing
    total_content_height = title_height + title_subtitle_spacing + subtitle_height
    start_y = top + (square_size - total_content_height) // 2

    # Position for title
    title_x = left + (square_size - title_width) // 2
    title_y = start_y

    # Position for subtitle
    subtitle_x = left + (square_size - subtitle_width) // 2
    subtitle_y = title_y + title_height + title_subtitle_spacing

    # Draw the text
    draw.multiline_text(
        (title_x, title_y),
        title,
        font=title_font,
        fill="white",
        align="center"
    )

    draw.multiline_text(
        (subtitle_x, subtitle_y),
        subtitle,
        font=subtitle_font,
        fill="white",
        align="center"
    )

    # Merge overlay with background and save
    final_image = Image.alpha_composite(background, overlay)
    final_image.save(output_path)


# from anthropic import Anthropic

# def translate_text(input_string, target_language):
#     """Translate text using Anthropic's Claude."""
#     try:
#         # Initialize the Anthropic client
#         anthropic = Anthropic(api_key=ANTHROPIC_API_KEY)  # Replace with your Anthropic API key
        
#         # Prepare the prompt
#         prompt = f"\n\nHuman: Translate the following text to {target_language}: {input_string}\n\nAssistant:"
        
#         # Call the Anthropic API for translation
#         response = anthropic.messages.create(
#             model="claude-3-opus-20240229",  # or "claude-2.1" depending on your needs
#             max_tokens=1000,
#             messages=[{
#                 "role": "user",
#                 "content": f"Translate the following text to {target_language}: {input_string}. if already in {target_language}, return {input_string} as it is. No Preamble"
#             }]
#         )
        
#         # Extract the translated text from the response
#         translated_text = response.content[0].text.strip()
#         return translated_text
        
#     except Exception as e:
#         print(f"Translation error: {str(e)}")
#         return input_string  # Return original string if translation fails
def translate_text(input_string: str, target_language: str, max_length=30) -> str:
    """
    Use Perplexity to detect language and translate if needed.
    Returns original text if source and target languages match.
    """
    api_key = os.getenv("PERPLEXITY_API_KEY")
    if not api_key:
        raise ValueError("PERPLEXITY_API_KEY environment variable not set")

    url = "https://api.perplexity.ai/chat/completions"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    # First, detect the language
    detect_payload = {
        "model": "llama-3.1-sonar-small-128k-online",
        "messages": [
            {
                "role": "system",
                "content": "You are a language detector. Respond with only the language . one word output. No additional text. without any special charaters including fullstop."
            },
            {
                "role": "user",
                "content": f"What language is this text in: {input_string}"
            }
        ]
    }

    try:
        detect_response = requests.post(url, json=detect_payload, headers=headers)
        detect_response.raise_for_status()
        source_language = detect_response.json()['choices'][0]['message']['content'].strip().lower()

        # If source and target languages match, return original
        if source_language == target_language.lower():
            return input_string

        # If languages differ, translate
        translate_payload = {
            "model": "llama-3.1-sonar-small-128k-online",
            "messages": [
                {
                    "role": "system",
                    "content": (
                        f"You are a translator, a very lazy translator, who just translates exact given input strings, not additional responses. Translate the following text to {target_language}. "
                        "Provide ONLY the direct translation of the exact text. "
                        "Do not add any explanations or additional content. "
                        "Do not expand or elaborate on the original text."
                    )
                },
                {
                    "role": "user",
                    "content": f"Translate this text exactly as is: {input_string}. Don't use explain the text. respond strictly in {target_language}, keep your response strictly under {max_length} characters."
                }
            ]
        }

        translate_response = requests.post(url, json=translate_payload, headers=headers)
        translate_response.raise_for_status()
        return translate_response.json()['choices'][0]['message']['content'].strip()

    except Exception as e:
        print(f"Error: {e}")
        return input_string

def beautify(input_string, language ,max_length=15):

    input_string=translate_text(input_string,language)
    words = input_string.split()  # Split the string into words
    lines = []  # To hold the final lines
    current_line = []  # To construct the current line
    
    for word in words:
        # If adding the next word exceeds the max_length, finalize the current line
        if sum(len(w) for w in current_line) + len(current_line) + len(word) > max_length:
            lines.append(" ".join(current_line).title())  # Capitalize each word in the line
            current_line = [word]  # Start a new line with the current word
        else:
            current_line.append(word)
    
    # Append the last line if it's not empty
    if current_line:
        lines.append(" ".join(current_line).title())  # Capitalize each word in the line
    
    return "\n".join(lines)  # Join the lines with \n

def extract_and_format_number(input_string):
    import re

    # Extract the leading number using regex
    match = re.match(r"(\d+)", input_string)
    if match:
        num = int(match.group(1))  # Extract the number and convert to integer
        formatted = f"{num}-{num + 1}"
        return formatted, num
    else:
        raise ValueError("The input string does not start with a number.")
    
from bs4 import BeautifulSoup, Comment

def extract_post_details(html_file_path):
    """
    Extract title, meta, and HTML content from a file.
    Returns a dictionary containing these details along with a generated slug.
    """
    try:
        with open(html_file_path, "r", encoding="utf-8") as file:
            html_content = file.read()

        # Parse the HTML content with BeautifulSoup
        soup = BeautifulSoup(html_content, "html.parser")

        # Extract title
        title_tag = soup.title
        title = title_tag.string.strip() if title_tag else "No title found"

        # Generate slug from title
        slug = title.lower().replace(' ', '-')
        slug = ''.join(c for c in slug if c.isalnum() or c == '-')

        # Extract meta content
        meta_contents = " ".join([
            tag.get("content", "")
            for tag in soup.find_all("meta")
            if tag.get("content") and "width=device-width" not in tag.get("content", "")
        ])

        # Extract HTML content using the original method
        start_index = html_content.find("<!-- wp:heading -->")
        if start_index == -1:
            raise ValueError("The starting tag '<!-- wp:heading -->' was not found.")

        end_index = html_content.find("</html>", start_index)
        if end_index == -1:
            end_index = len(html_content)

        html_snippet = html_content[start_index:end_index]

        return {
            "title": title,
            "slug": slug,
            "meta": meta_contents,
            "content": html_snippet
        }

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return {
            "title": "",
            "slug": "",
            "meta": "",
            "content": ""
        }

def create_zip_file(directory):
    memory_file = BytesIO()
    with zipfile.ZipFile(memory_file, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(directory):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, directory)
                zipf.write(file_path, arcname)
    memory_file.seek(0)
    return memory_file
def save_html_details_to_file(details: Dict, language: str, title: str, output_file: str):
    """Save HTML details to a text file with proper formatting."""
    with open(output_file, 'a', encoding='utf-8') as file:
        file.write(f"\n{'='*80}\n")
        file.write(f"Language: {language}\n")
        file.write(f"Title: {title}\n")
        file.write(f"{'='*80}\n\n")
        
        file.write("META CONTENTS:\n")
        file.write(f"{'-'*50}\n")
        file.write(f"{details['meta_contents']}\n\n")
        
        file.write("HTML SNIPPET:\n")
        file.write(f"{'-'*50}\n")
        file.write(f"{details['html_snippet']}\n\n")
        
        file.write(f"{'#'*80}\n\n")

# def main():
#     course_df = pd.read_csv('course_data_full.csv')
#     #### get title,subtitle, course_code from 


#     title = beautify(title,language)

#     subtitle = beautify(subtitle,language)
#     course_df=pd.read_csv('course_data_full.csv')
#     course_dict=course_df.loc[(course_df.course_code==course_code) & (course_df.language==language)].iloc[0].to_dict()

#     website=course_dict['website']
#     course_name=course_dict['course_name']
#     course_link=course_dict['course_link']
#     results_blog_dict=get_random_values_from_string(course_dict['result_blog_dict'],4)
#     results_product_dict=get_random_values_from_string(course_dict['results_product_dict'],4)
#     inpiration_blog_dict=get_random_values_from_string(course_dict['inspiration_blog_dict'],4)

#     num_of_words_in_blog,num_of_images_in_blog,num_of_headings,blog_percent,promotion_percent,num_of_infographics,user_notes_for_image,number_of_case_studies,number_of_tables,num_of_outbound_links=fetch_blog_specifications()
#     num_of_images_in_blog,num_of_headings=extract_and_format_number(title)

#     blog_identifier=GenerateBlogStructure(title,
#                                         course_name,
#                                         course_link,
#                                         results_blog_dict,
#                                         results_product_dict,
#                                         num_of_words_in_blog,
#                                         num_of_images_in_blog,
#                                         num_of_headings,
#                                         blog_percent,
#                                         promotion_percent,
#                                         num_of_infographics,
#                                         number_of_case_studies,
#                                         inpiration_blog_dict,
#                                         number_of_tables,
#                                         num_of_outbound_links,language)

#     SaveImages(blog_identifier)
#     ProduceFinalOutput(blog_identifier,website)

#     with open(f'Blogs/Organization/json/{blog_identifier}.json', "r") as file:
#         data = json.load(file)

#     Content_id=list(data.keys())[0]


#     input_path = f'Blogs/Organization/images/{blog_identifier}/{Content_id}.png'
#     output_path = f'Blogs/Organization/images/{blog_identifier}/{Content_id}-header.png'



#     try:
#         result_image = create_simple_overlay(
#             image_path=input_path,
#             output_path=output_path,
#             title=title,
#             subtitle=subtitle  # Added subtitle parameter
#         )
#         print("Image created successfully!")
#     except Exception as e:
#         print(f"Error creating image: {str(e)}")

#     html_file_path = f"Blogs/Organization/html/final_output/{blog_identifier}.html"
#     details = extract_html_details(html_file_path)
#     print("Title:", details["title"])
#     print("Meta Contents:", details["meta_contents"])
#     print("HTML Snippet:", details["html_snippet"])

# if __name__ == "__main__":
#     main()

import requests
import os
import random

def find_relevant_tags(blog_title: str, tag_list: list) -> list:
    """Find 2-3 most relevant tags from a randomly sorted list for a given blog title."""
    
    # Create a copy of the tag list and shuffle it
    shuffled_tags = tag_list.copy()
    random.shuffle(shuffled_tags)
    
    # Get API key from environment variable
    api_key = os.getenv("PERPLEXITY_API_KEY")
    if not api_key:
        raise ValueError("PERPLEXITY_API_KEY environment variable not set")

    # API endpoint
    url = "https://api.perplexity.ai/chat/completions"

    # Headers with API key
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    # Create the prompt with shuffled tags
    prompt = f"""Given the blog title: "{blog_title}"
    And this list of tags: {', '.join(shuffled_tags)}
    Return only 2-3 most relevant tags that best match the blog title's topic.
    Provide the response as a comma-separated list without explanations."""

    # Payload for the API
    payload = {
        "model": "llama-3.1-sonar-huge-128k-online",
        "messages": [
            {
                "role": "system",
                "content": "You are a content categorization expert. Provide direct, concise responses with only the most relevant tags."
            },
            {
                "role": "user",
                "content": prompt
            }
        ]
    }

    try:
        # Make the API request
        response = requests.post(url, json=payload, headers=headers)
        response.raise_for_status()

        # Extract the suggested tags
        result = response.json()
        suggested_tags = result['choices'][0]['message']['content'].strip()
        
        # Convert the comma-separated string to list and clean up
        tag_recommendations = [tag.strip() for tag in suggested_tags.split(',')]
        
        # Ensure we return only tags that exist in the original tag_list
        final_tags = [tag for tag in tag_recommendations if tag in tag_list][:3]
        
        return final_tags

    except requests.exceptions.RequestException as e:
        print(f"An error occurred while calling the Perplexity API: {e}")
        return []
    except KeyError as e:
        print(f"Unexpected response structure: {e}")
        return []

import requests
from requests.auth import HTTPBasicAuth
import os


def upload_image(image_path):
    headers = {
        "Authorization": f"Basic {username}:{password}"
    }
    with open(image_path, "rb") as img_file:
        files = {"file": img_file}
        response = requests.post(
            f"{site_url}/media",
            auth=HTTPBasicAuth(username, password),
            files=files
        )
    if response.status_code == 201:
        return response.json().get("id")
    else:
        print("Failed to upload image:", response.json())
        return None

def get_or_create_tags(tags_list):
    tag_ids = []
    for tag in tags_list:
        response = requests.get(
            f"{site_url}/tags?search={tag}",
            auth=HTTPBasicAuth(username, password)
        )
        if response.status_code == 200 and response.json():
            tag_ids.append(response.json()[0]["id"])
        else:
            response = requests.post(
                f"{site_url}/tags",
                auth=HTTPBasicAuth(username, password),
                json={"name": tag}
            )
            if response.status_code == 201:
                tag_ids.append(response.json()["id"])
    return tag_ids

def get_or_create_categories(categories_list):
    category_ids = []
    for category in categories_list:
        response = requests.get(
            f"{site_url}/categories?search={category}",
            auth=HTTPBasicAuth(username, password)
        )
        if response.status_code == 200 and response.json():
            category_ids.append(response.json()[0]["id"])
        else:
            response = requests.post(
                f"{site_url}/categories",
                auth=HTTPBasicAuth(username, password),
                json={"name": category}
            )
            if response.status_code == 201:
                category_ids.append(response.json()["id"])
    return category_ids

def get_user_id_by_username(username):
    response = requests.get(
        f"{site_url}/users?search={username}",
        auth=HTTPBasicAuth(username, password)
    )
    if response.status_code == 200:
        users = response.json()
        if users:
            return users[0]['id']
    return 1  # Default to user ID 1 if not found

def create_wordpress_post(details, blog_identifier, Tags, Category):
    print("creating word press post")
    print(details)
    # Get post details from the details dictionary
    post_title = details['title']
    html_content = details['content']
    post_slug = details['slug']
    excerpt = details['meta']

    print("details extracted")

    # Prepare Rank Math metadata
    rank_math_meta = {
        "rank_math_title": post_title,
        "rank_math_focus_keyword": ", ".join(Tags),  # Using tags as keywords
        "rank_math_permalink": post_slug
    }

    print("rank extracted")

    # Upload images and find header image
    image_ids = []
    featured_image_id = None
    image_dir = f'Blogs/Organization/images/{blog_identifier}'
    
    if os.path.exists(image_dir):
        for image_file in os.listdir(image_dir):
            if image_file.endswith(('.png', '.jpg', '.jpeg')):
                image_path = os.path.join(image_dir, image_file)
                image_id = upload_image(image_path)
                
                if image_id:
                    # If the image filename contains 'header', set it as featured image
                    if 'header' in image_file.lower():
                        featured_image_id = image_id
                    image_ids.append(image_id)


    print("header image found")

    # Prepare the post data
    author_id = 3

    # Prepare the post data
    post_data = {
        "title": post_title,
        "content": html_content,
        "slug": post_slug,
        "status": "draft",
        "excerpt": excerpt,
        "author": author_id,  # Using the fetched user ID
        "tags": get_or_create_tags(Tags),
        "categories": get_or_create_categories(Category),
        "meta": rank_math_meta,
    }
    # Set featured image if header image was found
    if featured_image_id:
        post_data["featured_media"] = featured_image_id
    elif image_ids:  # Fallback to first image if no header image found
        post_data["featured_media"] = image_ids[0]


    print("post data created")

    # Create the post
    response = requests.post(
        f"{site_url}/posts",
        auth=HTTPBasicAuth(username, password),
        json=post_data
    )

    print("posting done")

    if response.status_code == 201:
        print("Post created successfully!")
        print("Post ID:", response.json().get("id"))
        if featured_image_id:
            print("Header image set as thumbnail successfully!")
        else:
            print("Warning: No header image found to set as thumbnail")
    else:
        print("Failed to create post:", response.json())

def title_check_func(blog_identifier):
    with open(f'Blogs/Organization/html/blog_structure/{blog_identifier}.html', "r", encoding="utf-8") as file:
            html_content = file.read()
    soup = BeautifulSoup(html_content, "html.parser")

    # Extract title
    title_tag = soup.title
    title_check = title_tag.string.strip() if title_tag else "No title found"

    if title_check=="No title found":
        return True
    else:
        return False

def process_topic(topic: Dict, language: str, max_retry=3):
    """Process a single topic for a specific language."""
    try:
        course_df = pd.read_csv('course_data_full.csv')
        
        title = beautify(topic['title'], language)
        subtitle = beautify(topic['subtitle'], language)
        course_code = topic['course_code']

        course_dict = course_df.loc[
            (course_df.course_code == course_code) & 
            (course_df.language == language)
        ].iloc[0].to_dict()
        # WordPress site details



        website = course_dict['website']
        course_name = course_dict['course_name']
        course_link = course_dict['course_link']

        Category = course_dict['Category'].split('|')
        Tags = course_dict['Tags'].split('|')

        Tags=find_relevant_tags(title,Tags)
        
        results_blog_dict = get_random_values_from_string(course_dict['result_blog_dict'], 4)
        results_product_dict = get_random_values_from_string(course_dict['results_product_dict'], 4)
        inpiration_blog_dict = get_random_values_from_string(course_dict['inspiration_blog_dict'], 4)

        num_of_words_in_blog,num_of_images_in_blog,num_of_headings,blog_percent,promotion_percent,num_of_infographics,user_notes_for_image,number_of_case_studies,number_of_tables,num_of_outbound_links=fetch_blog_specifications()
        num_of_images_in_blog, num_of_headings = extract_and_format_number(title)

        global site_url, username, password
        site_url = f"https://{website}.com/wp-json/wp/v2"
        username = "rishabh.deepak.shukla@gmail.com"
        

        # reading the data from the file 
        with open('password.txt') as f: 
            data = f.read() 

        passwords= ast.literal_eval(data) 
        password = passwords[language]

        blog_identifier=GenerateBlogStructure(title,
                                            course_name,
                                            course_link,
                                            results_blog_dict,
                                            results_product_dict,
                                            num_of_words_in_blog,
                                            num_of_images_in_blog,
                                            num_of_headings,
                                            blog_percent,
                                            promotion_percent,
                                            num_of_infographics,
                                            number_of_case_studies,
                                            inpiration_blog_dict,
                                            number_of_tables,
                                            num_of_outbound_links,language)
        

        count=0
        while title_check_func(blog_identifier):
            
            if count==max_retry:
                raise f"unable to create blog for  {title} in {language}"
            print(f"title not found retrying ({count+1})..")
            count+=1
            blog_identifier=GenerateBlogStructure(title,
                                    course_name,
                                    course_link,
                                    results_blog_dict,
                                    results_product_dict,
                                    num_of_words_in_blog,
                                    num_of_images_in_blog,
                                    num_of_headings,
                                    blog_percent,
                                    promotion_percent,
                                    num_of_infographics,
                                    number_of_case_studies,
                                    inpiration_blog_dict,
                                    number_of_tables,
                                    num_of_outbound_links,language)

        SaveImages(blog_identifier)
        ProduceFinalOutput(blog_identifier, website)

        # Process the blog JSON
        json_path = f'Blogs/Organization/json/{blog_identifier}.json'
        with open(json_path, "r") as file:
            data = json.load(file)

        Content_id = list(data.values())[0].split('/')[-1].split('.')[0]

        # Process images
        input_path = f'Blogs/Organization/images/{blog_identifier}/{Content_id}.png'
        output_path = f'Blogs/Organization/images/{blog_identifier}/{Content_id}-header.png'



        try:
            create_simple_overlay(
                image_path=input_path,
                output_path=output_path,
                title=title,
                subtitle=subtitle
            )
            print(f"Image created successfully for {language} - {title}")
        except Exception as e:
            print(f"Error creating image for {language} - {title}: {str(e)}")


        # Extract HTML details
        html_file_path = f"Blogs/Organization/html/final_output/{blog_identifier}.html"
        details = extract_post_details(html_file_path)

        print("details found")
        
        # Save details to file
        # save_html_details_to_file(details, language, title, output_file)
        
        create_wordpress_post(details, blog_identifier, Tags, Category)
        
        print(f"Processed {language} - {title}")
        
    except Exception as e:
        print(f"Error processing {language} - {title}: {str(e)}")

# def process_topic(topic: Dict, language: str, output_file: str):
#     """Process a single topic for a specific language."""
#     try:
#         course_df = pd.read_csv('course_data_full.csv')
        
#         title = beautify(topic['title'], language)
#         subtitle = beautify(topic['subtitle'], language)
#         course_code = topic['course_code']

#         course_dict = course_df.loc[
#             (course_df.course_code == course_code) & 
#             (course_df.language == language)
#         ].iloc[0].to_dict()

#         website = course_dict['website']
#         course_name = course_dict['course_name']
#         course_link = course_dict['course_link']
        
#         results_blog_dict = get_random_values_from_string(course_dict['result_blog_dict'], 4)
#         results_product_dict = get_random_values_from_string(course_dict['results_product_dict'], 4)
#         inpiration_blog_dict = get_random_values_from_string(course_dict['inspiration_blog_dict'], 4)

#         num_of_words_in_blog,num_of_images_in_blog,num_of_headings,blog_percent,promotion_percent,num_of_infographics,user_notes_for_image,number_of_case_studies,number_of_tables,num_of_outbound_links=fetch_blog_specifications()
#         num_of_images_in_blog, num_of_headings = extract_and_format_number(title)

#         blog_identifier=GenerateBlogStructure(title,
#                                             course_name,
#                                             course_link,
#                                             results_blog_dict,
#                                             results_product_dict,
#                                             num_of_words_in_blog,
#                                             num_of_images_in_blog,
#                                             num_of_headings,
#                                             blog_percent,
#                                             promotion_percent,
#                                             num_of_infographics,
#                                             number_of_case_studies,
#                                             inpiration_blog_dict,
#                                             number_of_tables,
#                                             num_of_outbound_links,language)

#         SaveImages(blog_identifier)
#         ProduceFinalOutput(blog_identifier, website)

#         # Process the blog JSON
#         json_path = f'Blogs/Organization/json/{blog_identifier}.json'
#         with open(json_path, "r") as file:
#             data = json.load(file)

#         Content_id = list(data.keys())[0]

#         # Process images
#         input_path = f'Blogs/Organization/images/{blog_identifier}/{Content_id}.png'
#         output_path = f'Blogs/Organization/images/{blog_identifier}/{Content_id}-header.png'

#         try:
#             create_simple_overlay(
#                 image_path=input_path,
#                 output_path=output_path,
#                 title=title,
#                 subtitle=subtitle
#             )
#             print(f"Image created successfully for {language} - {title}")
#         except Exception as e:
#             print(f"Error creating image for {language} - {title}: {str(e)}")

#         # Extract HTML details
#         html_file_path = f"Blogs/Organization/html/final_output/{blog_identifier}.html"
#         details = extract_html_details(html_file_path)
        
#         # Save details to file
#         save_html_details_to_file(details, language, title, output_file)
#         print(f"Processed {language} - {title}")
        
#     except Exception as e:
#         print(f"Error processing {language} - {title}: {str(e)}")
import requests
import os

def generate_subtitle(blog_title: str, language:str, length=20) -> str:
    """Generate a subtitle for a given blog title using Perplexity AI."""
    
    # Get API key from environment variable
    api_key = os.getenv("PERPLEXITY_API_KEY")
    if not api_key:
        raise ValueError("PERPLEXITY_API_KEY environment variable not set")

    # API endpoint
    url = "https://api.perplexity.ai/chat/completions"

    # Headers with API key
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    # Payload for subtitle generation
    payload = {
        "model": "llama-3.1-sonar-huge-128k-online",
        "messages": [
            {
                "role": "system",
                "content": (
                    f"you are a expert subtitle creator in {language}"
                    "Generate a concise and engaging subtitle for the given blog title. "
                    "The subtitle should be shorter than the title and complement its meaning. "
                    f"length of title is {len(blog_title)} hence subtile should less than {len(blog_title)}. ultimately less than 30 characters regardeless of the title length"
                    "Provide only the subtitle without any explanations.No preamble,No special characters to be returned including (\",:,full stop)"
                )
            },
            {
                "role": "user",
                "content": f"Generate a subtitle for this blog title: {blog_title}. the length of subtitle should be at max {length} characters which includes spaces."
            }
        ]
    }

    try:
        # Make the API request
        response = requests.post(url, json=payload, headers=headers)
        response.raise_for_status()

        # Extract the generated subtitle
        result = response.json()
        subtitle = result['choices'][0]['message']['content'].strip()

            
        return subtitle

    except requests.exceptions.RequestException as e:
        print(f"An error occurred while calling the Perplexity API: {e}")
        return "Error generating subtitle"
    except KeyError as e:
        print(f"Unexpected response structure: {e}")
        return "Error generating subtitle"

def main():
    """Main function to process all languages and topics."""
    # Create output file name with timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_file = f"html_details_{timestamp}.txt"
    
    # Add header to the output file
    with open(output_file, 'w', encoding='utf-8') as file:
        file.write("HTML DETAILS EXTRACTION REPORT\n")
        file.write(f"Generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        file.write(f"{'='*80}\n\n")
    
    # Read input file
    input_data = read_input_file('input_file1.txt')
    
    # Process each language and topic combination
    for language in input_data['language']:
        for topic in input_data['topic']:
            print(f"\nProcessing {language} - {topic['title']}")
            topic['subtitle']=generate_subtitle(topic['title'],language).replace("\"","")
            process_topic(topic, language, output_file)
    
    print(f"\nOutput has been saved to: {output_file}")

if __name__ == "__main__":
    main()


Processing Italian - 7 Creative Ways to Express Grief Through Art
Generated HTML ends with </html>
title not found retrying (1)..
Generated HTML ends with </html>
Saved image: Blogs/Organization/images/79031975/Peaceful_Artist_Studio_Painting_Easel_Supplies_Soft_Colors.png
Saved image: Blogs/Organization/images/79031975/Vibrant_Abstract_Emotion_Painting_on_Large_Canvas.png
Saved image: Blogs/Organization/images/79031975/Hands_Molding_Clay_Symbolic_Shape_Memory_Loss.png
Saved image: Blogs/Organization/images/79031975/Creative_Collage_Process_Magazine_Photos_Textures_Arrangement.png
Saved image: Blogs/Organization/images/79031975/Journaling_in_Candlelit_Study_with_Books_and_Pen.png
Saved image: Blogs/Organization/images/79031975/Photographer_capturing_meaningful_moment_in_warm_light.png
Saved image: Blogs/Organization/images/79031975/Colorful_Mandala_Artist_Creating_Design_with_Organized_Art_Supplies.png
Saved image: Blogs/Organization/images/79031975/Expressive_dancer_in_bright_spaciou